<a href="https://colab.research.google.com/github/lsiecker/Deep-Learning/blob/Task-1/assignment_2/Assignment_2_2AMM10_22_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group Details

## Group Name: group21

### Student 1: N.P.G.T. van Beuningen	1353624

### Student 2: D.P.M. van der Hoorn	1873334

### Student 3: L.R. Siecker	1344838

# Loading Data and Preliminaries

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import io
import requests
import torch

In [2]:
base_url = "https://github.com/lsiecker/Deep-Learning/raw/main/assignment_2/data/"

In [3]:
def load_data(url, task):
    """
    Loads a numpy array from surfdrive. 
    
    Input:
    url: Download link of dataset 
    
    Outputs:
    dataset: numpy array with input features or labels
    """
    
    response = requests.get(url)
    response.raise_for_status()

    array = load_array(io.BytesIO(response.content), task)

    return array

In [4]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [5]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_data(f"{base_url}task%201/train/trajectory_0.npz?raw=true", task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [6]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_data(f'{base_url}task%202_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_data(f'{base_url}task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-1.11611543  3.21149953         nan         nan]
 [-0.2865083   4.30801877         nan         nan]
 [ 1.07701594 -8.12529269         nan         nan]
 [-0.92053478  3.13709551         nan         nan]
 [-3.96308297 -4.27733589         nan         nan]
 [ 2.33945401 -8.67733599         nan         nan]
 [-4.83949085  3.67854952         nan         nan]
 [ 0.31080159 -9.74720071         nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


# Task **1**

## Data Handling and Preprocessing

In [7]:
""" Get all training data """
train_data = []
for i in range(0,900):
  train_data.append(load_data(f"{base_url}task%201/train/trajectory_{i}.npz?raw=true", task='task 1'))


In [8]:
""" Get all test data """
test_data = []
for i in range(900, 1000):
  test_data.append(load_data(f"{base_url}task%201/test/trajectory_{i}.npz?raw=true", task='task 1'))

In [9]:
""" Create training, validation and test sets """
train_x = [torch.tensor(array[0]) for array in train_data[:800]]
train_y = [torch.tensor(array[1]) for array in train_data[:800]]
val_x = [torch.tensor(array[0]) for array in train_data[800:]]
val_y = [torch.tensor(array[1]) for array in train_data[800:]]
test_x = [torch.tensor(array[0]) for array in test_data]
test_y = [torch.tensor(array[1]) for array in test_data]


In [10]:
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

train_x_pad = pad_sequence(train_x, batch_first=True, padding_value=0) #add padding since some some observations have more objects than others
train_y_pad = pad_sequence(train_y, batch_first=True, padding_value=0)
val_x_pad = pad_sequence(val_x, batch_first=True, padding_value=0)
val_y_pad = pad_sequence(val_y, batch_first=True, padding_value=0)
test_x_pad = pad_sequence(test_x, batch_first=True, padding_value=0)
test_y_pad = pad_sequence(test_y, batch_first=True, padding_value=0)

train_dataset = TensorDataset(train_x_pad, train_y_pad)
val_dataset = TensorDataset(val_x_pad, val_y_pad)
test_dataset = TensorDataset(test_x_pad, test_y_pad)

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

## Model Implementation

In [11]:
#todo

In [71]:
from torch import nn

class TrainPointPredictor(nn.Module):
    def __init__(self):
        pass

    def forward(self, points, padded_points):
        pass


In [24]:
import time

def train(dataloader):

  model.train()

  total_correct, total_count = 0,0
  log_interval = 500
  start_time = time.time()

  for idx, (trajectory_points, padded_points) in enumerate(dataloader):
    trajectory_points = trajectory_points.to(device)
    padded_points = padded_points.to(device)

    model.optimizer.zero_grad()

    y_pred = model(trajectory_points, padded_points)

    # TODO classes are kind of continuous in this task
    loss = model.criterion(y_pred.permute((0,2,1)), padded_points)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
    model.optimizer.step()
    
    total_correct += (y_pred.argmax(2) == padded_points).sum().item()
    total_count += padded_points.size(0) * 35
    
    
    if idx % log_interval == 0 and idx > 0:
        elapsed = time.time() - start_time
        print('| epoch {:3d} | {:5d}/{:5d} batches '
              '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                          total_correct/total_count))
        total_correct, total_count = 0, 0
        start_time = time.time()
            
    return total_correct/total_count

def evaluate(dataloader):
    model.eval()
    total_correct, total_count = 0, 0

    with torch.no_grad():
        for idx, (trajectory_points, padded_points) in enumerate(dataloader):
            
            trajectory_points = trajectory_points.to(device)
            padded_points = padded_points.to(device)
            
            y_pred = requests.models(trajectory_points, padded_points)
            
            # TODO: classes are kind of continuous in this case
            loss = model.criterion(..., padded_points)
            
            total_correct += (y_pred.argmax(2) == padded_points).sum().item()
            total_count += padded_points.size(0) * 35

    return total_correct/total_count

In [72]:
# HYPERPARAMETERS
EPOCHS = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TrainPointPredictor().to(device)

train_acc, val_acc = [], []
# training loop
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    train_acc.append(train(train_dataloader))
    val_acc.append(evaluate(val_dataloader))
    
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'train accuracy {:8.3f} '
          'validation accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time, 
                                           train_acc[-1],
                                           val_acc[-1]))
    print('-' * 59)

AttributeError: ignored

## Model Training

In [13]:
#todo

## Evaluation

In [14]:
#todo

# Task **2**

## Data Handling and Preprocessing

In [15]:
#todo

## Model Implementation

In [16]:
#todo

## Model Training

In [17]:
#todo

## Evaluation

In [18]:
#todo

# Task **3**

## Data Handling and Preprocessing

In [19]:
#todo

## Model Implementation

In [20]:
#todo

## Model Training

In [21]:
#todo

## Evaluation

In [22]:
#todo